### Steps in this stage
- Label encode city,pincode,region_code data
- Install `pip install tldextract`
- not needed anymore - Rehydrate referer_url from step 4
- Encode it with only domain+tld
- Label Encode new field `url`

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract

import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))
import util.utils as utils
import util.plot_utils as plot_utils



plt.rcParams["figure.figsize"] = (12,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score



InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2)

In [3]:
project_name="click-prediction"
def get_file_location(filename):
    return "../data/%s/%s" % (project_name,filename)

In [4]:
df_conv_status = pd.read_csv("../data/%s/%s" % (project_name,"train-true_6.csv"))
df_fraud= pd.read_csv("../data/%s/%s" % (project_name,"train-fraud_6.csv"))
df = pd.read_csv("../data/%s/%s" % (project_name,"train_6.csv"))
df_test= pd.read_csv("../data/%s/%s" % (project_name,"test_6.csv"))

/home/ahemf/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:


def analyse_by_payout_avg(df,colname):
    dfc=df.groupby(colname)

    dfav=pd.DataFrame(dfc['ConversionPayOut'].sum() / dfc["ID"].count())

    dfav.columns = ["pay"]
    dfav.sort_values("pay",ascending=False).head(10)

#     dfsum = pd.DataFrame(dfc['ConversionPayOut'].sum())
    dfcount = pd.DataFrame(dfc["ID"].count())
    dfcount.columns=["count"]
    print("done")
    return dfcount.sort_values("count",ascending=False).head(10)
    
    
    
analyse_by_payout_avg(df,"city")
analyse_by_payout_avg(df_conv_status,"city")
    

done


,count
city,
Dhaka,250015
Bangkok,97159
Jakarta,68337
Lagos,37405
Kuala Lumpur,32913
Delhi,31980
Kolkata,26389
Moscow,25224
Bengaluru,24287


done


,count
city,
Dhaka,4334
Lagos,778
Jakarta,639
Bangkok,616
Moscow,263
Port Harcourt,254
Kuwait City,225
Johannesburg,179
Mexico City,124


In [5]:
analyse_by_payout_avg(df,"region_code")
analyse_by_payout_avg(df_conv_status,"region_code")

done


,count
region_code,
C,266861
10,151410
84,103641
JK,80643
TN,65045
MH,62550
WB,55060
UP,46026
DL,43229


done


,count
region_code,
C,4372
84,904
10,870
LA,780
JK,751
JT,327
GT,301
JI,282
MOW,263


In [7]:
def label_encode_field(df1,df2,df3,df_test,field):
    df1[field] = df1[field].astype(str)
    df2[field] = df2[field].astype(str)
    df3[field] = df3[field].astype(str)
    df_test[field] = df_test[field].astype(str)
    
    df1[field] = df1[field].fillna('-997')
    df2[field] = df2[field].fillna('-997')
    df3[field] = df3[field].fillna('-997')
    
    df_test[field] = df_test[field].fillna('-997')
    
    encoded_df = df1[field].append(df2[field]).append(df3[field]).append(df_test[field])
    encoded_df.fillna('-997')
    label_encoder = LabelEncoder()
    encoder = label_encoder.fit(encoded_df.values)
    encoded_t=encoder.transform(df_test[field].fillna("-997").values)
    df_test[field] = encoded_t
    
    encoded_df1=encoder.transform(df1[field].fillna("-997").values)
    df1[field] = encoded_df1
    
    encoded_df2=encoder.transform(df2[field].fillna("-997").values)
    df2[field] = encoded_df2
    
    encoded_df3=encoder.transform(df3[field].fillna("-997").values)
    df3[field] = encoded_df3
    
    return encoder


def store_encoder_as_file(le,column_name):
    ids =np.arange(0,len(list(le.classes_))).astype(int)
    my_encoding = pd.DataFrame(list(le.classes_), ids, columns = [column_name])
    print(my_encoding.shape)
    my_encoding.to_csv(get_file_location("%s-encoding.csv" % column_name), index_label = ["id"])

In [8]:
df.dtypes


ID                        int64
Carrier                   int64
TrafficType               int64
Device                    int64
Browser                   int64
OS                        int64
RefererUrl                int64
ConversionPayOut        float64
publisherId               int64
subPublisherId            int64
advertiserCampaignId      int64
Fraud                     int64
conv_status               int64
dayofweek                 int64
hour                      int64
dayofyear                 int64
network                   int64
subnet                    int64
city                     object
region_code              object
postal_code              object
Country                 float64
dtype: object

In [9]:
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"city"),"city_step-6")
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"region_code"),"region_code-6")
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"postal_code"),"postal_code-6")

(51131, 1)
(808, 1)
(54144, 1)


In [10]:
def domain(url):
    r = extract(url)
    res="-1"
    if (len(r.domain) > 0) | (len(r.suffix)>0):
        res = r.domain +"."+ r.suffix
    return res

def sub_domain(url):
    r = extract(url)
    res="-1"
    if (len(r.subdomain)>0 & ((len(r.domain) > 0) | (len(r.suffix)>0))):
        res = r.subdomain + "."+ r.domain +"."+ r.suffix
    return res


In [11]:
df_refurl = pd.read_csv("../data/%s/%s" % (project_name,"RefererUrl-encoding.csv"))

In [12]:

df_refurl["sub_domain"]=df_refurl["RefererUrl"].map(sub_domain)    
df_refurl["url"]=df_refurl["RefererUrl"].map(domain)

In [14]:
df_refurl.tail()
df_refurl.head()
df_refurl["url"].fillna("-997",inplace=True)
df_refurl["sub_domain"].fillna("-997",inplace=True)
df_refurl.tail()
df_refurl.head()

,id,RefererUrl,sub_domain,url
404281,404281,zznew.in,-1,zznew.in
404282,404282,zznzz.alladu.kidx.gdn,zznzz.alladu.kidx.gdn,kidx.gdn
404283,404283,zzvzz.alladu.kidx.gdn,zzvzz.alladu.kidx.gdn,kidx.gdn
404284,404284,zzvzz.gcpxamse.dama.gdn,zzvzz.gcpxamse.dama.gdn,dama.gdn
404285,404285,zzyzz.alladu.kidx.gdn,zzyzz.alladu.kidx.gdn,kidx.gdn


,id,RefererUrl,sub_domain,url
0,0,"(function(pa,s){if(document.getElementById('pr...","(function(pa,s){if(document.getElementById('pr...",getElementById('pr4dd307a9'))return false;.
1,1,(null),-1,(null).
2,2,-997,-1,-997.
3,3,/03J17/-CcjAA/7CRm/-TV1/umwjALbneg0any-_xaU2Tz...,-1,-1
4,4,/03J17/-CcjAA/7CRm/-TV1/umwjALbnel1MnSu_zfI1Sd...,-1,-1


,id,RefererUrl,sub_domain,url
404281,404281,zznew.in,-1,zznew.in
404282,404282,zznzz.alladu.kidx.gdn,zznzz.alladu.kidx.gdn,kidx.gdn
404283,404283,zzvzz.alladu.kidx.gdn,zzvzz.alladu.kidx.gdn,kidx.gdn
404284,404284,zzvzz.gcpxamse.dama.gdn,zzvzz.gcpxamse.dama.gdn,dama.gdn
404285,404285,zzyzz.alladu.kidx.gdn,zzyzz.alladu.kidx.gdn,kidx.gdn


,id,RefererUrl,sub_domain,url
0,0,"(function(pa,s){if(document.getElementById('pr...","(function(pa,s){if(document.getElementById('pr...",getElementById('pr4dd307a9'))return false;.
1,1,(null),-1,(null).
2,2,-997,-1,-997.
3,3,/03J17/-CcjAA/7CRm/-TV1/umwjALbneg0any-_xaU2Tz...,-1,-1
4,4,/03J17/-CcjAA/7CRm/-TV1/umwjALbnel1MnSu_zfI1Sd...,-1,-1


In [15]:
# For easy merge
df_refurl["RefererUrl"]  = df_refurl["id"]
df_refurl.drop(['id'], axis = 1, inplace = True)


In [16]:
# # Not needed since we kept refurl
# df_conv_status_4 = pd.read_csv("../data/%s/%s" % (project_name,"train-true_4.csv"))
# df_fraud_4= pd.read_csv("../data/%s/%s" % (project_name,"train-fraud_4.csv"))
# df_4 = pd.read_csv("../data/%s/%s" % (project_name,"train_4.csv"))
# df_test_4= pd.read_csv("../data/%s/%s" % (project_name,"test-step-2.csv"))

In [64]:


# dft=df.merge(df_refurl,on="RefererUrl",how="left")
# dft.shape
# dft[~(dft["url"]=="-1.")].shape
# dft.head()

# len(dft["url"].unique())

(34524, 19)

In [17]:
def add_url(df,df_refurl):
    dft=df.merge(df_refurl,on="RefererUrl",how="left")
    return dft

df_conv_status = add_url(df_conv_status,df_refurl)
df_fraud= add_url(df_fraud,df_refurl)
df = add_url(df,df_refurl)

gc.collect()

    

61

In [18]:
df_test = add_url(df_test,df_refurl)

In [20]:
df.tail()

len(df.dtypes)
len(df_conv_status.dtypes)
len(df_test.dtypes)


,ID,Carrier,TrafficType,Device,Browser,OS,RefererUrl,ConversionPayOut,publisherId,subPublisherId,...,hour,dayofyear,network,subnet,city,region_code,postal_code,Country,sub_domain,url
38020325,61790414,-1,0,388,2,1,2,0.0,5250,0,...,18,242,13450,614554,51056,807,54143,218.0,-1,-997.
38020326,61996496,1,0,388,38,1,2,0.0,1079,0,...,19,242,1215,71578,51056,807,54143,99.0,-1,-997.
38020327,63138047,-1,2,601,1,1,2,0.0,678,0,...,23,242,25833,1185862,51056,807,54143,3.0,-1,-997.
38020328,61152759,-1,2,388,2,1,391312,0.0,439,0,...,16,242,10880,394667,51056,807,54143,115.0,m.dcinside.com,dcinside.com
38020329,63294549,1,0,388,38,1,2,0.0,1079,0,...,23,242,1213,71161,51056,742,54143,99.0,-1,-997.


24

24

22

466

In [21]:
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"url"),"url-step_6")
store_encoder_as_file(label_encode_field(df_conv_status,df_fraud,df,df_test,"sub_domain"),"sub_domain-step_6")

(12246, 1)
(13114, 1)


In [22]:
df.to_csv("train_7.csv",index=False)
df_conv_status.to_csv("train-true_7.csv",index=False)
df_fraud.to_csv("train-fraud_7.csv",index=False)
df_test.to_csv("test_7.csv",index=False)

### ConversionCapping
- Cap payout at always >=0
- Cap payout between 0-100, if train data has errors but test data is proper then its logical that payouts cant be over 100 for any click
- Test both payout <100 and non capped model for submission

In [23]:
len(df_test["ID"].unique())

25548873

In [24]:
len(df["ID"].unique())

38020330